# Decision Trees

Decision trees are versitile ML aglorithms that can perform classification, regression and multioutput tasks. Decision trees are also fundamentla components of Random Forests, which are among the most powerful ML aglorithms today.

We will discuss how to train, visualize and make predictions. then we will go through CART trainng algorithm used by Scikit-Learn, and we will discuss how to regularise trees and use them for regression tasks. We will also discuss the limitations

Lets bult a decision tree using a DecisionTreeClassifier on the iris dataset

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # Petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [2]:
# we use the export_graphviz method to output a graph definition file called iris_tree.dot

from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

# then we can use the dot command line tool from Graphviz package ti convert .dot image to png/pdf etc.
# dot -Tpng iris_tree.dot -o iris_tree.png


<img src="./iris_tree.png">